In [4]:
import pandas as pd
import numpy as np
import datetime as dt

In [5]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.style.use('ggplot')
# %matplotlib inline
# np.set_printoptions(suppress=True)

In [84]:
df = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2019-01.csv')

In [85]:
df.head(1)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.0,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1,NaN


In [86]:
df.columns = df.columns.str.lower()

## Payment Types
Removing payment type 2, cash payments. The data source noted that cash rides do not record tips

In [87]:
df.payment_type.value_counts()

1    388313
2    237857
3      3392
4      1330
5        26
Name: payment_type, dtype: int64

In [88]:
df = df[df.payment_type !=2]

In [89]:
# mapping the numerical values into their appropriate titles since this is categorical

payment_dict = {1:'credit',
               3:'no charge',
               4:'Dispute',
               5:'unknown',
               6:'voided'}

In [90]:
df.payment_type = df.payment_type.map(payment_dict)

In [91]:
df.head()

,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pulocationid,dolocationid,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,credit,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,credit,1,NaN
5,2,2019-01-01 00:12:35,2019-01-01 00:19:09,N,1,49,17,1,1.05,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,credit,1,NaN
6,2,2019-01-01 00:47:55,2019-01-01 01:00:01,N,1,255,33,1,3.77,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,credit,1,NaN
7,1,2019-01-01 00:12:47,2019-01-01 00:30:50,N,1,76,225,1,4.10,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,credit,1,NaN


## Removing Columns
Some columns only existed for yellow, or green cars. Removing them to combine later. Also remvoing "congestion_surcharge" There were a large number of NaN's in both data sets. A baseline tree consisting of non-NaN congestion surcharge also showed little gain from this feature

In [92]:
df.drop(columns=['ehail_fee','trip_type','store_and_fwd_flag',
                 'congestion_surcharge','vendorid'],inplace=True)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393061 entries, 2 to 630917
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   vendorid               393061 non-null  int64  
 1   lpep_pickup_datetime   393061 non-null  object 
 2   lpep_dropoff_datetime  393061 non-null  object 
 3   ratecodeid             393061 non-null  int64  
 4   pulocationid           393061 non-null  int64  
 5   dolocationid           393061 non-null  int64  
 6   passenger_count        393061 non-null  int64  
 7   trip_distance          393061 non-null  float64
 8   fare_amount            393061 non-null  float64
 9   extra                  393061 non-null  float64
 10  mta_tax                393061 non-null  float64
 11  tip_amount             393061 non-null  float64
 12  tolls_amount           393061 non-null  float64
 13  improvement_surcharge  393061 non-null  float64
 14  total_amount           393061 non-nu

## Setting Target

Changing the "tip_amount" column to "tip" and changing it to a binary column to use as my target

In [94]:
df.rename(columns = {'tip_amount':'tip'},inplace=True)

In [95]:
#putting target variable in first column

cols = list(df.columns)
cols.insert(0, cols.pop(cols.index('tip')))
df = df[cols]

In [96]:
def to_binary(value):
    if value > 0:
        return 1
    else:
        return 0

In [97]:
df.tip = df.tip.apply(to_binary)

## Ratecode ID

converting the ratecode ID numerical values to appropriate labels for one hot encoding

In [98]:
ratecode_dict = {1:'standard',
                2:'jfk',
                3:'newark',
                4:'nassau/weschester',
                5:'negotiated',
                6:'group'}

In [99]:
df.ratecodeid = df.ratecodeid.map(ratecode_dict)

In [100]:
df.ratecodeid.value_counts()

standard             346466
negotiated            45693
jfk                     485
nassau/weschester       290
newark                  124
group                     3
Name: ratecodeid, dtype: int64

## Datetime Elements

Using the provided timestamps, I am creating features for week of the month, day of the week, and hour of the day, all stored as string values to later be one-hot-encoded

In [101]:
#making the columns easier to work with

df.rename(columns = {'lpep_pickup_datetime':'pickup_datetime','lpep_dropoff_datetime':'dropoff_datetime'},inplace=True)

In [102]:
df.pickup_datetime, df.dropoff_datetime = pd.to_datetime(df.pickup_datetime), pd.to_datetime(df.dropoff_datetime)

In [103]:
df['week_of_month'] = df.pickup_datetime.dt.weekofyear

# removing erroneous dates for accuracy

df = df[df.week_of_month<6]

# converting column to string

df.week_of_month = df.week_of_month.astype(str)

<ipython-input-103-c27da1b06245>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_month'] = df.pickup_datetime.dt.weekofyear


In [104]:
df['pickup_hour'] = (df.pickup_datetime.dt.hour).astype(str)
df['dropoff_hour'] = (df.dropoff_datetime.dt.hour).astype(str)
df['day'] = (df.pickup_datetime.dt.day_name()).astype(str)

In [105]:
df.head(1)

,tip,vendorid,pickup_datetime,dropoff_datetime,ratecodeid,pulocationid,dolocationid,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,payment_type,week_of_month,pickup_hour,dropoff_hour,day
2,0,2,2019-01-01 00:27:11,2019-01-01 00:31:38,standard,49,189,2,0.66,4.5,0.5,0.5,0.0,0.3,5.8,credit,1,0,0,Tuesday


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393050 entries, 2 to 630917
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tip                    393050 non-null  int64         
 1   pickup_datetime        393050 non-null  datetime64[ns]
 2   dropoff_datetime       393050 non-null  datetime64[ns]
 3   ratecodeid             393050 non-null  object        
 4   pulocationid           393050 non-null  int64         
 5   dolocationid           393050 non-null  int64         
 6   passenger_count        393050 non-null  int64         
 7   trip_distance          393050 non-null  float64       
 8   fare_amount            393050 non-null  float64       
 9   extra                  393050 non-null  float64       
 10  mta_tax                393050 non-null  float64       
 11  tolls_amount           393050 non-null  float64       
 12  improvement_surcharge  393050 non-null  floa

## Borough Information

pulling in a separate dataframe to change location id's into Borough information

In [110]:
locationdf = pd.read_csv('data/locations.csv')

In [111]:
locationdf

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [112]:
locationdf.columns = locationdf.columns.str.lower()

In [113]:
#merging for pickup information
df.rename(columns={'pulocationid':'locationid'}, inplace=True)
df = pd.merge(df, locationdf[['locationid','borough','zone']], how='left', on='locationid')

#renaming columns
df.rename(columns={'borough':'pickup_borough', 'locationid':'pickup_location_id',
                  'dolocationid':'locationid','zone':'pickup_zone'}, inplace=True)

#merging for dropoff information
df = pd.merge(df, locationdf[['locationid','borough','zone']], how='left', on='locationid')
df.rename(columns={'borough':'dropoff_borough','locationid':'dropoff_location_id',
                  'zone':'dropoff_zone'},inplace=True)

In [114]:
df.head(1)

,tip,pickup_datetime,dropoff_datetime,ratecodeid,pickup_location_id,dropoff_location_id,passenger_count,trip_distance,fare_amount,extra,...,total_amount,payment_type,week_of_month,pickup_hour,dropoff_hour,day,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,0,2019-01-01 00:27:11,2019-01-01 00:31:38,standard,49,189,2,0.66,4.5,0.5,...,5.8,credit,1,0,0,Tuesday,Brooklyn,Clinton Hill,Brooklyn,Prospect Heights


## Car Color

Creating a column that will signify if the car is a yellow or green taxi for when the data is combined

In [145]:
df['car_type'] = 'green'

In [147]:
cols = list(df.columns)
cols.insert(1, cols.pop(cols.index('car_type')))
df = df[cols]

In [150]:
df.to_csv(r'/users/michaelharnett/desktop/metis/projects/taxi_tip_classification_metis4/data/green1.csv', index=False)

In [118]:
df.shape

(393050, 23)

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393050 entries, 0 to 393049
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tip                    393050 non-null  int64         
 1   car_type               393050 non-null  object        
 2   pickup_datetime        393050 non-null  datetime64[ns]
 3   dropoff_datetime       393050 non-null  datetime64[ns]
 4   ratecodeid             393050 non-null  object        
 5   pickup_location_id     393050 non-null  int64         
 6   dropoff_location_id    393050 non-null  int64         
 7   passenger_count        393050 non-null  int64         
 8   trip_distance          393050 non-null  float64       
 9   fare_amount            393050 non-null  float64       
 10  extra                  393050 non-null  float64       
 11  mta_tax                393050 non-null  float64       
 12  tolls_amount           393050 non-null  floa

In [162]:
object_list = list(df.select_dtypes('object').columns)
#object_list.pop(object_list.index('car_type'))

In [163]:
object_list

['car_type',
 'ratecodeid',
 'payment_type',
 'week_of_month',
 'pickup_hour',
 'dropoff_hour',
 'day',
 'pickup_borough',
 'pickup_zone',
 'dropoff_borough',
 'dropoff_zone']

In [120]:
# dummiesdf = pd.get_dummies(data = df, columns=['ratecodeid','payment_type','week_of_month',
#                                              'pickup_hour','dropoff_hour','day','pickup_borough',
#                                              'pickup_zone','dropoff_borough','dropoff_zone'], drop_first=True)

In [164]:
dummiesdf = pd.get_dummies(data = df, columns=object_list[1:], drop_first = True)

In [165]:
dummiesdf.shape

(393050, 600)

In [166]:
dummiesdf.head()

,tip,car_type,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id,passenger_count,trip_distance,fare_amount,extra,...,dropoff_zone_Williamsbridge/Olinville,dropoff_zone_Williamsburg (North Side),dropoff_zone_Williamsburg (South Side),dropoff_zone_Windsor Terrace,dropoff_zone_Woodhaven,dropoff_zone_Woodlawn/Wakefield,dropoff_zone_Woodside,dropoff_zone_World Trade Center,dropoff_zone_Yorkville East,dropoff_zone_Yorkville West
0,0,green,2019-01-01 00:27:11,2019-01-01 00:31:38,49,189,2,0.66,4.5,0.5,...,0,0,0,0,0,0,0,0,0,0
1,1,green,2019-01-01 00:46:20,2019-01-01 01:04:54,189,17,2,2.68,13.5,0.5,...,0,0,0,0,0,0,0,0,0,0
2,0,green,2019-01-01 00:12:35,2019-01-01 00:19:09,49,17,1,1.05,6.5,0.5,...,0,0,0,0,0,0,0,0,0,0
3,0,green,2019-01-01 00:47:55,2019-01-01 01:00:01,255,33,1,3.77,13.5,0.5,...,0,0,0,0,0,0,0,0,0,0
4,0,green,2019-01-01 00:12:47,2019-01-01 00:30:50,76,225,1,4.10,16.0,0.5,...,0,0,0,0,0,0,0,0,0,0


In [167]:
dummiesdf.to_csv(r'/users/michaelharnett/desktop/metis/projects/taxi_tip_classification_metis4/data/green_dummies.csv', index=False)